In [1]:
import numpy as np
from numpy import random
import collections
import time

# Parametros del modelo

In [2]:
# Información problema
S=4       #capacidad max y tamaño de lote
a=1       #Demanda minima
b=2       #Demanda maxima
cost=300  #costo de la cerveza
k=100     #costo de realizar el pedido  
h=300     #costo de inventario sobrante
price=1500#precio cerveza 


tamano_L=3 # se fdefine para hacer pruebas

#### Generó las demandas

In [3]:
random.seed(10)
total=4# cantidad de periodos o semanas
D=random.randint(a,b+1, size=(total)) #emanda para cada uno de los 4 periodos
D

array([2, 2, 1, 2])

In [4]:
def reward(i,D_i):
        dem=D_i
        r_dem=price*min(dem,i)
        if(dem>=i):
            r_dem+=-(price-cost)*(dem-i) #costo perdida clientes
        else:
            #if(dem<i):#Agrego costo de inventario sobrante
            r_dem+= -(h)*(i-dem)
        r =(r_dem)
        return r

In [5]:
reward(3,D[0])#D_i demanda en en periodo i

2700

In [6]:
def retorno_decision(d,tamano_L,S,D_i,i):
    #d es la decision compro o no compro
    # i es el nivel de inventario al inicio del periodo
    #S=4       #capacidad max y tamaño de lote
    #costo=300  #precio cerveza y costo perdida clientes
    #k=100     #costo de realizar el pedido  
    #price=1500
    
    B=0 #retorno
    cant_comprada_i=tamano_L*d
    if cant_comprada_i>0:
    #Juan decide comprar mas cerveza.
        if(cant_comprada_i+i<=S):
        #La compra más el inventario debe ser menor a Cap max
            B=reward(cant_comprada_i+i,D_i)
            B+=-cost*(cant_comprada_i)-k
        else:
            B+=-cost*(cant_comprada_i)
            # Regalo las cervezas que no puedo guardar
    else:
        B=reward(i,D_i)
    return(B)
    

In [7]:
retorno_decision(0,tamano_L,S,D[1],3)

2700

In [8]:
def Retorno_lote(tamano_L,S,D,):
    l=len(D)
    Decion=[[ 0 for i in range(S)]for i in range(l)]
    R_D=[[ 0 for i in range(S)]for i in range(l)]
    for i in range(l-1,-1,-1):#periodo desde el ultimo periodo al anterior
        for s in range(S):#niveles de inventario posibles
            D_i=D[i]
            D_1=(retorno_decision(1,tamano_L,S,D_i,i))
            D_0=retorno_decision(0,tamano_L,S,D_i,i)
            if(i<l-1):
                D_1 +=R_D[i+1][max(S-D_i,0)]
                D_0 +=R_D[i+1][max(S-D_i,0)]
            if(D_1>D_0):
                Decion[i][s]=1
                R_D[i][s]=D_1
            else:
                Decion[i][s]=0
                R_D[i][s]=D_0
    I=[0 for i in range(l)]
    R=0
    DECIS=[0 for i in range(l)]
    for i in range(l-1):#periodo
        s=I[i]
        DECIS[i]=Decion[i][s]
        R+=R_D[i][s]
        J=max(0,s+Decion[i][s]*tamano_L-D[i])
        I[i+1]=min(J,S)
    return R

In [ ]:
def Retorno_lote(tamano_L,S,D,):
    l=len(D)
    Decion=[[ 0 for i in range(S)]for i in range(l)]
    R_D=[[ 0 for i in range(S)]for i in range(l)]
    I=[0 for i in range(l)]
    R=0
    DECIS=[0 for i in range(l)]
    for i in range(l-1,-1,-1):#periodo desde el ultimo periodo al anterior
        for s in range(S):#niveles de inventario posibles
            D_i=D[i]
            D_1=(retorno_decision(1,tamano_L,S,D_i,i))
            D_0=retorno_decision(0,tamano_L,S,D_i,i)
            if(i<l-1):
                D_1 +=R_D[i+1][max(S-D_i,0)]
                D_0 +=R_D[i+1][max(S-D_i,0)]
            if(D_1>D_0):
                Decion[i][s]=1
                R_D[i][s]=D_1
            else:
                Decion[i][s]=0
                R_D[i][s]=D_0
    I=[0 for i in range(l)]
    R=0
    DECIS=[0 for i in range(l)]
    for i in range(l-1):#periodo
        s=I[i]
        DECIS[i]=Decion[i][s]
        R+=R_D[i][s]
        J=max(0,s+Decion[i][s]*tamano_L-D[i])
        I[i+1]=min(J,S)
    return R

In [9]:
Retorno_lote(tamano_L,S,D)

16200

In [28]:
[Retorno_lote(i,S,D) for i in range(0,S)]

[9900, 16800, 18000, 16200, 14400, 12800]

In [27]:
#Fuerza bruta
S=6
A=[Retorno_lote(i,S,D) for i in range(0,S)]
print('El tamaño de lote es ',A.index(max(A)),'con una utilidad ',max(A))
    

El tamaño de lote es  2 con una utilidad  18000


In [26]:
#Aplicación con busqueda local, cuando el tamaño del frigobar es muy grande
S=6
for i in range(1,6):
   if(Retorno_lote(i,S,D)<Retorno_lote(i-1,S,D)):
    print('El tamaño de lote es ',i-1,'con una utilidad ',Retorno_lote(i-1,S,D))
    break


El tamaño de lote es  2 con una utilidad  18000
